<a href='https://www.darshan.ac.in/'> <img src='https://www.darshan.ac.in/Content/media/DU_Logo.svg' width="250" height="300"/></a>
<pre>
<center><b><h1>Data Mining</b></center>
<center><b><h1>Lab - 7 (Part 2)</b></center>    
    <center><b><h1>23010101226 - Drashti Rathod - 516</b></center>   

<pre>

### Step 1: Load the Dataset
Load the `Tdata.csv` file and display the first few rows.

In [1]:
import pandas as pd
import numpy as np

td = pd.read_csv("Tdata.csv")
td

,Transaction,bread,butter,coffee,eggs,jam,milk
0,T1,1,1,0,0,0,1
1,T2,1,1,0,0,1,0
2,T3,1,0,0,1,0,1
3,T4,1,1,0,0,0,1
4,T5,1,0,1,0,0,0
5,T6,0,0,1,1,1,0


### Step 2: Drop the 'Transaction' Column
We're only interested in the items (not the transaction IDs).

In [3]:
td.drop("Transaction",axis=1,inplace=True)
td

,bread,butter,coffee,eggs,jam,milk
0,1,1,0,0,0,1
1,1,1,0,0,1,0
2,1,0,0,1,0,1
3,1,1,0,0,0,1
4,1,0,1,0,0,0
5,0,0,1,1,1,0


### Step 3: Count Single Items
See how many transactions include each item.

In [5]:
td.sum()

bread     5
butter    3
coffee    2
eggs      2
jam       2
milk      3
dtype: int64

### Step 4: Define Apriori Function
This function finds frequent itemsets of size 1, 2, and 3 with minimum support.

In [7]:
from itertools import combinations

def find_frequent_itemsets(td, min_support):
    n = len(td)
    result = []
    for k in [1,2,3]: #for 1-item, 2-item, 3-item sets
        for items in combinations(td.columns, k):
            mask = td[list(items)].all(axis=1)
            #print("items", items, "mask", mask.sum())
            support = mask.sum()/n;
            if support >= min_support:
                result.append((frozenset(items), round(support,2)))
    return result

### Step 5: Run Apriori
Set `min_support = 0.6` and display the frequent itemsets.

In [9]:
frequent_itemsets = find_frequent_itemsets(td,min_support = 0.5)
for itemset, support in frequent_itemsets:
    print(f"{set(itemset)} → support: {support}")


{'bread'} → support: 0.83
{'butter'} → support: 0.5
{'milk'} → support: 0.5
{'butter', 'bread'} → support: 0.5
{'milk', 'bread'} → support: 0.5


In [13]:
for itemset, support in frequent_itemsets:
    if itemset == frozenset(['bread']):
        print(f"{set(itemset)} → support: {support}")

{'bread'} → support: 0.83


### Step 6 Display as a DataFrame

In [11]:
result_td = pd.DataFrame(frequent_itemsets, columns=['Itemset','Support'])
result_td

,Itemset,Support
0,(bread),0.83
1,(butter),0.50
2,(milk),0.50
3,"(butter, bread)",0.50
4,"(milk, bread)",0.50


# Orange Tool : - >Generate Same Frequent Patterns in Orange tools

In [ ]:
# Done - Screenshots

# Extra : - >  Define Apriori Function without itertools

## Step 1: Convert DataFrame to list of transactions


In [64]:
transactions = []
for _, row in td.iterrows():
    transaction = [item for item in td.columns if row[item] == 1]
    transactions.append(transaction)

## Step 2: Apriori implementation (without itertools)

### Step 2.1: Get unique items

In [66]:
def get_unique_items(transactions):
    items = []
    for transaction in transactions:
        for item in transaction:
            if item not in items:
                items.append(item)
    return sorted(items)


### Step 2.2: Generate candidates (k-item combinations)

In [68]:
def generate_candidates(prev_frequent_itemsets, k):
    candidates = []
    for i in range(len(prev_frequent_itemsets)):
        for j in range(i + 1, len(prev_frequent_itemsets)):
            itemset1 = prev_frequent_itemsets[i]
            itemset2 = prev_frequent_itemsets[j]
            union = sorted(list(set(itemset1) | set(itemset2)))
            if len(union) == k and union not in candidates:
                candidates.append(union)
    return candidates


### Step 2.3: Count support of Itemsets

In [76]:
def count_support(itemsets, transactions, min_support):
    counts = {}
    for itemset in itemsets:
        count = 0
        for transaction in transactions:
            if all(item in transaction for item in itemset):
                count += 1
        support = count / len(transactions)
        if support >= min_support:
            counts[tuple(itemset)] = support
    return counts


### Step 2.4: Apriori loop

In [82]:
def apriori(transactions, min_support):
    unique_items = get_unique_items(transactions)
    current_itemsets = [[item] for item in unique_items]  # L1
    frequent_itemsets = []
    k = 1

    while current_itemsets:
        support_counts = count_support(current_itemsets, transactions, min_support)
        current_frequent = list(support_counts.keys())
        frequent_itemsets.extend(support_counts.items())
        k += 1
        current_itemsets = generate_candidates(current_frequent, k)

    return frequent_itemsets

### Step 2.5: Print Result

In [84]:
results = apriori(transactions, min_support=0.5)

for itemset, support in results:
    print(f"{list(itemset)}: {support:.2f}")

['bread']: 0.83
['butter']: 0.50
['milk']: 0.50
['bread', 'butter']: 0.50
['bread', 'milk']: 0.50
